<a href="https://colab.research.google.com/github/frankxiao008/30-Days-of-Python/blob/master/MU_Monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium webdriver-manager yfinance pandas beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
# 1. Install the specialized Colab Selenium package
!pip install google-colab-selenium



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.0 MB/s eta 0:00:00


In [ ]:
# 2. Use the package to initialize the driver
import google_colab_selenium as gs
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install google-colab-selenium yfinance pandas beautifulsoup4

In [ ]:


import google_colab_selenium as gs
import yfinance as yf
import pandas as pd
import time
import os
from datetime import datetime
from bs4 import BeautifulSoup

class MUMemoryMonitor:
    def __init__(self):
        self.ticker_mu = "MU"
        self.ticker_nanya = "2408.TW"
        self.eps_2026_projection = 35.50
        self.log_file = "mu_cycle_history.csv"


    def get_spot_prices(self):
        print("🌐 Scraping latest memory spot prices...")

        driver = gs.Chrome()
        prices = {"DDR4": 62.38, "DDR5": 28.22} # Default fallbacks

        try:
            driver.get("https://www.dramexchange.com/")
            time.sleep(10) # Give more time for cloud rendering
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            rows = soup.find_all('tr')

            for row in rows:
                cols = row.find_all('td')
                # Safety check: Row must have at least 6 columns (Price is in index 5)
                if len(cols) < 6: continue

                text = row.text.replace(" ", "")
                price_str = cols[5].text.strip() # The "Session Avg" column

                # Helper to check if string is a number
                def is_number(s):
                    try:
                        float(s)
                        return True
                    except ValueError:
                        return False

                if is_number(price_str):
                    current_price = float(price_str)

                    # Target specific chips
                    if "DDR416Gb(2Gx8)3200" in text:
                        prices["DDR4"] = current_price
                        print(f"✅ Found DDR4: {current_price}")
                    elif "DDR516G(2Gx8)4800/5600" in text:
                        prices["DDR5"] = current_price
                        print(f"✅ Found DDR5: {current_price}")

        except Exception as e:
            print(f"⚠️ Scraping Issue: {e}. Using fallback values.")
        finally:
            driver.quit()
        return prices

    def log_and_analyze(self, spot_data):
        mu = yf.Ticker(self.ticker_mu)
        mu_price = mu.fast_info['last_price']
        nanya = yf.Ticker(self.ticker_nanya).fast_info['last_price']

        inversion_ratio = spot_data['DDR4'] / spot_data['DDR5']
        forward_pe = mu_price / self.eps_2026_projection

        entry = {
            "Date": datetime.now().strftime('%Y-%m-%d %H:%M'),
            "MU_Price": round(mu_price, 2),
            "DDR4_Spot": spot_data['DDR4'],
            "DDR5_Spot": spot_data['DDR5'],
            "Inversion_Ratio": round(inversion_ratio, 3),
            "Forward_PE": round(forward_pe, 2),
            "Nanya_TWD": round(nanya, 2)
        }

        df = pd.DataFrame([entry])
        if not os.path.isfile(self.log_file):
            df.to_csv(self.log_file, index=False)
        else:
            df.to_csv(self.log_file, mode='a', index=False, header=False)

        return entry

    def display_report(self, data):
        print(f"\n{'='*50}")
        print(f" MICRON STRATEGY REPORT - {data['Date']}")
        print(f"{'='*50}")
        print(f"1. INVERSION RATIO: {data['Inversion_Ratio']}")
        status = "✅ BULLISH (Shortage Active)" if data['Inversion_Ratio'] >= 1.0 else "⚠️ WARNING (Normalizing)"
        print(f"   Status: {status}")
        print(f"2. VALUATION: ${data['MU_Price']} (P/E: {data['Forward_PE']}x)")
        print(f"3. LAGGARD (Nanya): TWD {data['Nanya_TWD']}")
        print(f"{'='*50}\nMetrics logged to {self.log_file}")

# EXECUTION
monitor = MUMemoryMonitor()
current_spots = monitor.get_spot_prices()
latest_data = monitor.log_and_analyze(current_spots)
monitor.display_report(latest_data)

🌐 Scraping latest memory spot prices...


<IPython.core.display.Javascript object>

✅ Found DDR5: 28.333
✅ Found DDR4: 63.564

 MICRON STRATEGY REPORT - 2025-12-30 06:07
1. INVERSION RATIO: 2.243
   Status: ✅ BULLISH (Shortage Active)
2. VALUATION: $294.37 (P/E: 8.29x)
3. LAGGARD (Nanya): TWD 194.5
Metrics logged to mu_cycle_history.csv
